<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/9_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import time
import multiprocessing

app = dash.Dash(__name__)
server = app.server

step_names = ["撈取資料", "進行 Rule Check", "繪製圖片", "上拋資料"]

# ---- 建立預設進度 ----
def default_progress():
    return {
        'current_index': -1,
        'step_status': ['pending'] * len(step_names),
        'step_times': [None] * len(step_names),
        'done': False,
        'error': False
    }

# ---- 報告流程 ----
def run_report_process(progress, stop_event):
    try:
        start_time = time.time()

        # step 1
        progress['current_index'] = 0
        progress['step_status'][0] = 'running'
        if stop_event.is_set(): return
        time.sleep(2)
        progress['step_status'][0] = 'success'
        progress['step_times'][0] = int(time.time() - start_time)

        # step 2
        progress['current_index'] = 1
        progress['step_status'][1] = 'running'
        if stop_event.is_set(): return
        time.sleep(2)
        progress['step_status'][1] = 'success'
        progress['step_times'][1] = int(time.time() - start_time)

        # step 3
        progress['current_index'] = 2
        progress['step_status'][2] = 'running'
        if stop_event.is_set(): return
        time.sleep(2)
        progress['step_status'][2] = 'success'
        progress['step_times'][2] = int(time.time() - start_time)

        # step 4
        progress['current_index'] = 3
        progress['step_status'][3] = 'running'
        if stop_event.is_set(): return
        time.sleep(2)
        progress['step_status'][3] = 'success'
        progress['step_times'][3] = int(time.time() - start_time)

        progress['done'] = True

    except Exception:
        progress['error'] = True
        progress['step_status'][progress['current_index']] = 'error'


# ---- 全域 Process Pool ----
manager = multiprocessing.Manager()
progress = manager.dict(default_progress())
stop_event = multiprocessing.Event()
proc = None


# ---- Layout ----
app.layout = html.Div([
    html.H3("報告流程 (multiprocessing 版本)"),
    html.Button("Report", id="run-btn"),
    html.Button("Kill", id="kill-btn", style={"marginLeft": "10px"}),
    html.Div(id="step-list", style={'display': 'none'}),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True)
])


# ---- 開始流程 ----
@app.callback(
    Output("run-btn", "disabled"),
    Output("interval", "disabled"),
    Output("step-list", "style"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_process(n):
    global proc, progress, stop_event
    # 如果舊的還在，先殺掉
    if proc and proc.is_alive():
        stop_event.set()
        proc.terminate()
        proc.join()

    # 重置狀態
    progress = manager.dict(default_progress())
    stop_event = multiprocessing.Event()

    # 啟動子程序
    proc = multiprocessing.Process(target=run_report_process, args=(progress, stop_event))
    proc.start()

    return True, False, {"marginTop": "20px", "fontSize": "16px"}


# ---- 中止流程 ----
@app.callback(
    Output("step-list", "style", allow_duplicate=True),
    Output("run-btn", "disabled", allow_duplicate=True),
    Output("interval", "disabled", allow_duplicate=True),
    Input("kill-btn", "n_clicks"),
    prevent_initial_call=True
)
def kill_process(n):
    global proc, stop_event
    stop_event.set()
    if proc and proc.is_alive():
        proc.terminate()
        proc.join()
    return {'display': 'none'}, False, True


# ---- 更新 UI ----
@app.callback(
    Output("step-list", "children"),
    Input("interval", "n_intervals")
)
def update_ui(n):
    elements = []
    for i, step in enumerate(step_names):
        status = progress['step_status'][i]
        color = {'pending': 'gray', 'running': 'orange', 'success': 'green', 'error': 'red'}.get(status, 'gray')
        symbol = {'pending': "□", 'running': "▶", 'success': "✔", 'error': "❌"}.get(status, "□")
        time_spent = f" ({progress['step_times'][i]}秒)" if progress['step_times'][i] else ""
        elements.append(html.Div(f"{symbol} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))
    return elements


if __name__ == "__main__":
    app.run(debug=True, threaded=False)  # 關閉多執行緒
